In [24]:
# scales(PHQ9_total  GAD7_total STAI_X1_total)- MaDE
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

In [25]:
import pandas as pd
import numpy as np
import os

In [26]:
df_labelled = pd.read_csv('./YAD/Labelled+webmini.csv')
print(df_labelled)
print(type(df_labelled))

     Unnamed: 0        ID  Gender  Age  PHQ9_total  GAD7_total STAI_X1_total  \
0             0  YAD30211       1   27          14           9            47   
1             1  YAD30504       1   24           3           3            33   
2             2  YAD30211       1   26          22          12            47   
3             3  YAD30212       1   24          22          10            29   
4             4  YAD30213       1   28           8           3            36   
..          ...       ...     ...  ...         ...         ...           ...   
289         289  MDD20019       1   24          10           8            64   
290         290  MDD20020       0   25          10          14            71   
291         291  MDD20021       0   31          17          12            62   
292         292  MDD20022       0   20          14          11         empty   
293         293  MDD20023       1   25           4           0            36   

    LSAS_performance_fear LSAS_situatio

In [27]:
df_unlabelled_K = pd.read_csv('./YAD/Unlabelled_KAIST.csv')
print(df_unlabelled_K)

          No                                        ID  \
0        494  438550c7ec70a9ba62a114e93586f6c4f2490cb2   
1        501  09f679c6c3be3da2ba3a6b1f09cfc582a1fd138b   
2        505  a73b7ab092df5a28b7ea79e6803a876332fe13dc   
3        510  72f26d01617b27d594f3c31647d88732014c010c   
4        511  38857c6812e0f37b74db69a9f0910cb3a5b933a2   
...      ...                                       ...   
17247  30490  840842512161278f5da5628a4f4accd5834fa1ed   
17248  30491  c474c86376cd13ba738e36dbf58a09789666ed0b   
17249  30492  7819ea09468d5e7d78746728bee2392c70c5ba59   
17250  30494  56a4001930985c21ef45a8436a77c6e55833f0b6   
17251  30497  f3dc642ddf1b0ab1030d08c19a188086a24bc01a   

                                           ID_2 appointment_date  Age  Gender  \
0      54efb2832e5ca9e8b299ce827c112a1c965c14e2       2017-03-24   24       1   
1      4b012a3cb9d1866e62bf12c018a05e7896ff5b80       2017-03-24   29       1   
2      139991fcc7a5504432014eebaaed80479fb37682       2017-0

C:\Users\CNDLMembers\anaconda3\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (92) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
print(df_unlabelled_K.columns)

Index(['No', 'ID', 'ID_2', 'appointment_date', 'Age', 'Gender', 'Grade',
       'SMOKING_01', 'SMOKING_cigar', 'SMOKING_yrs',
       ...
       'KSSI_1y_11', 'KSSI_1y_12', 'KSSI_1y_13', 'KSSI_1y_14', 'KSSI_1y_Total',
       'KSSI_atpt_1y', 'KSSI_atpt_life', 'KSSI_atpt_life_no',
       'KSSI_atpt_life_plan', 'KSSI_atpt_life_impul'],
      dtype='object', length=291)


In [29]:
print(df_labelled['PHQ9_total'].dtypes)
print(df_labelled['STAI_X1_total'].dtypes)

int64
object


In [30]:
df_labelled['STAI_X1_total'] = pd.to_numeric(df_labelled['STAI_X1_total'], errors='coerce') 

print(df_labelled['STAI_X1_total'].dtypes)

float64


In [31]:
train_df_labelled = df_labelled[df_labelled['site'].isin(['KAIST', 'Gachon'])]
test_df_labelled = df_labelled[df_labelled['site'] == 'SMC']
valid_df_labelled = df_labelled[df_labelled['site'] == 'SNU']

In [32]:
train_x = train_df_labelled.loc[:, ['PHQ9_total', 'GAD7_total', 'STAI_X1_total']]
test_x = test_df_labelled.loc[:, ['PHQ9_total', 'GAD7_total', 'STAI_X1_total']]
valid_x = valid_df_labelled.loc[:, ['PHQ9_total', 'GAD7_total', 'STAI_X1_total']]

print(train_x)

     PHQ9_total  GAD7_total  STAI_X1_total
0            14           9           47.0
1             3           3           33.0
2            22          12           47.0
3            22          10           29.0
4             8           3           36.0
..          ...         ...            ...
134           2           5           45.0
135           9           2           29.0
136           4           2           42.0
137          15          14           65.0
138           6           2           44.0

[139 rows x 3 columns]


In [33]:
train_y = train_df_labelled.loc[:, 'MaDE'].to_frame()
test_y = test_df_labelled.loc[:, 'MaDE'].to_frame()
valid_y = valid_df_labelled.loc[:, 'MaDE'].to_frame()

print(train_y)

     MaDE
0       1
1       0
2       1
3       1
4       1
..    ...
134     0
135     0
136     0
137     0
138     0

[139 rows x 1 columns]


In [34]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torch.nn.functional as F
import torch.optim as optim

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [36]:
x_train = torch.from_numpy(train_x.values)
print(x_train.shape)
print(x_train.type())
y_train = torch.from_numpy(train_y.values)
print(y_train.shape)
print(y_train.type())

torch.Size([139, 3])
torch.DoubleTensor
torch.Size([139, 1])
torch.LongTensor


In [37]:
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))
hypothesis = torch.sigmoid(x_train.float().matmul(W) + b)

In [48]:
losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis) )
cost = losses.mean()

F.binary_cross_entropy(hypothesis, y_train.float())  #loss와 cost 따로 정의 안해도 한번에 cost 함수 정의해줌

tensor(0.5147, grad_fn=<BinaryCrossEntropyBackward>)

In [39]:
optimizer = torch.optim.SGD([W, b], lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)

optimizer.zero_grad()
cost.backward()
optimizer.step()

In [40]:
nb_epochs = 10000
for epoch in range(1, nb_epochs + 1):
    hypothesis = torch.sigmoid(x_train.float().matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train.float())
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost, accuracy * 100
        ))
  

Epoch  100/10000 Cost: 0.302783 Accuracy 90.65%
Epoch  200/10000 Cost: 0.257963 Accuracy 91.37%
Epoch  300/10000 Cost: 0.241786 Accuracy 91.37%
Epoch  400/10000 Cost: 0.233764 Accuracy 91.37%
Epoch  500/10000 Cost: 0.229082 Accuracy 91.37%
Epoch  600/10000 Cost: 0.226056 Accuracy 91.37%
Epoch  700/10000 Cost: 0.223957 Accuracy 91.37%
Epoch  800/10000 Cost: 0.222421 Accuracy 91.37%
Epoch  900/10000 Cost: 0.221252 Accuracy 91.37%
Epoch 1000/10000 Cost: 0.220333 Accuracy 91.37%
Epoch 1100/10000 Cost: 0.219594 Accuracy 91.37%
Epoch 1200/10000 Cost: 0.218987 Accuracy 91.37%
Epoch 1300/10000 Cost: 0.218481 Accuracy 91.37%
Epoch 1400/10000 Cost: 0.218055 Accuracy 91.37%
Epoch 1500/10000 Cost: 0.217692 Accuracy 91.37%
Epoch 1600/10000 Cost: 0.217380 Accuracy 91.37%
Epoch 1700/10000 Cost: 0.217111 Accuracy 91.37%
Epoch 1800/10000 Cost: 0.216878 Accuracy 91.37%
Epoch 1900/10000 Cost: 0.216674 Accuracy 91.37%
Epoch 2000/10000 Cost: 0.216496 Accuracy 91.37%
Epoch 2100/10000 Cost: 0.216340 Accuracy

In [41]:
prediction = hypothesis >= torch.FloatTensor([0.5])

print(prediction)
print(y_train)
correct_prediction = prediction.float() == y_train
print(correct_prediction)

tensor([[ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [False

In [42]:
import torch
from torch.autograd import Variable

class BinaryClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
model = BinaryClassifier()

In [43]:
optimizer = torch.optim.SGD(model.parameters(),lr=1e-5)

In [44]:
nb_epochs = 10000
for epoch in range(1, nb_epochs + 1):
    hypothesis = model(x_train.float())
    cost = F.binary_cross_entropy(hypothesis, y_train.float())
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        prediction = hypothesis.float() >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train.float()
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost, accuracy * 100
        ))

Epoch  100/10000 Cost: 5.603292 Accuracy 23.02%
Epoch  200/10000 Cost: 4.874562 Accuracy 21.58%
Epoch  300/10000 Cost: 4.198388 Accuracy 17.99%
Epoch  400/10000 Cost: 3.586376 Accuracy 17.27%
Epoch  500/10000 Cost: 3.049006 Accuracy 16.55%
Epoch  600/10000 Cost: 2.598665 Accuracy 15.83%
Epoch  700/10000 Cost: 2.244958 Accuracy 11.51%
Epoch  800/10000 Cost: 1.987314 Accuracy 14.39%
Epoch  900/10000 Cost: 1.811569 Accuracy 19.42%
Epoch 1000/10000 Cost: 1.695841 Accuracy 21.58%
Epoch 1100/10000 Cost: 1.619416 Accuracy 23.74%
Epoch 1200/10000 Cost: 1.567024 Accuracy 28.06%
Epoch 1300/10000 Cost: 1.528828 Accuracy 32.37%
Epoch 1400/10000 Cost: 1.498912 Accuracy 34.53%
Epoch 1500/10000 Cost: 1.473828 Accuracy 34.53%
Epoch 1600/10000 Cost: 1.451587 Accuracy 43.17%
Epoch 1700/10000 Cost: 1.431046 Accuracy 43.17%
Epoch 1800/10000 Cost: 1.411543 Accuracy 43.88%
Epoch 1900/10000 Cost: 1.392696 Accuracy 44.60%
Epoch 2000/10000 Cost: 1.374285 Accuracy 44.60%
Epoch 2100/10000 Cost: 1.356179 Accuracy

In [45]:
###Test
x_test = torch.from_numpy(test_x.values)
print(x_test.shape)
print(x_test.type())
y_test = torch.from_numpy(test_y.values)
print(y_test.shape)
print(y_test.type())


torch.Size([91, 3])
torch.DoubleTensor
torch.Size([91, 1])
torch.LongTensor


In [46]:
model_t = PModel()

criterion = torch.nn.MSELoss(size_average=False) 
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

for epoch in range(501):
    y_pred =model(x_test.float())
    loss = criterion(y_pred, y_test.float()) 
    if(epoch%10==0):
        print(epoch, loss.item())
    
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step() 

NameError: name 'PModel' is not defined